# Aerial Imagery Classifier

Sam Blake, started 23 December 2022

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook
%pylab notebook

import os
import time
import glob
import math
import numpy as np
import cv2
import matplotlib as mpl
from matplotlib import pyplot as plt

from tqdm import tqdm

import numba
from numba import jit, prange, set_num_threads

from PIL import Image

import rasterio
from rasterio.plot import show

from aerial_imagery_classifier import imagery_statistical_binary_classification, \
    show_bgr_image, show_bgra_image, show_greyscale_image, denoise 

plt.ioff()
plt.rcParams["figure.figsize"] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


## De Bortolli Rice

In [41]:
tile_42_crop_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/crop_1.png')
tile_42_dirt_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/dirt_1.png')
tile_42_water_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/water_1.png')
tile_42_weed_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/weed_1.png')

In [42]:
tile_47_crop_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_1.png')
tile_47_crop_2 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_2.png')
tile_47_crop_3 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_3.png')
tile_47_dead_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/dead_1.png')
tile_47_weed_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/weed_1.png')
tile_47_weed_2 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/weed_2.png')

In [48]:
filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_42.tif'
filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_47.tif'
filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_52.tif'

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [tile_47_weed_1, tile_47_weed_2, tile_42_weed_1], \
        [tile_47_crop_1, tile_47_crop_2, tile_47_crop_3, tile_47_dead_1, \
            tile_42_dirt_1, tile_42_water_1], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (7,7), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 1, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.75, 3], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 175, morph_close = 10, \
        dilation_size = 301, \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)


# Export spray map to GeoTIFF.
output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*spray_region)

# Export weed map to GeoTIFF.
output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

# Export classification CI to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, classification_confidence)

In [ ]:
file_pattern = '/Users/user/Documents/Research/rotavision/de_bortolli/DeBortolli/map/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [tile_47_weed_1, tile_47_weed_2, tile_42_weed_1], \
            [tile_47_crop_1, tile_47_crop_2, tile_47_crop_3, tile_47_dead_1, \
                tile_42_dirt_1, tile_42_water_1], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (7,7), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 1, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.75, 3], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 175, morph_close = 10, \
            dilation_size = 301, \
            half_normal_dist = True, \
            export_histogram = False, \
            export_heatmap = False, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, 255*spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

result_tile_34.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0009
   31762 [px] in 2.75 CI, 0.0411 [%] of all pixels. 0.0411 [%] cumulative total.
   15995 [px] in 3.00 CI, 0.0207 [%] of all pixels. 0.0618 [%] cumulative total.
Sample not detected in 99.94 [%] of all pixels.
Spray region is 2.42 [%] of total region.
result_tile_20.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0015
   61434 [px] in 2.75 CI, 0.0795 [%] of all pixels. 0.0795 [%] cumulative total.
   27073 [px] in 3.00 CI, 0.0350 [%] of all pixels. 0.1146 [%] cumulative total.
Sample not detected in 99.89 [%] of all pixels.
Spray region is 4.22 [%] of total region.
result_tile_21.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0014
    5592 [px] in 2.75 CI, 0.0072 [%] of all pixels. 0.0072 [%] cumulative

/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
result_tile_22.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540


## TavistockAG Precision Weed Trial

In [31]:
dry_crop_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_1.jpg')
dry_crop_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_2.jpg')
dry_crop_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_3.jpg')
dry_crop_4 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_4.jpg')
dry_crop_5 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_5.jpg')
dry_crop_6 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_6.jpg')
dry_crop_7 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_7.jpg')

dry_crop_8 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_8.jpg')
rocks_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_1.jpg')
rocks_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_2.jpg')
rocks_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_3.jpg')
rocks_4 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_4.jpg')
rocks_5 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_5.jpg')

In [32]:
weed_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_1.jpg')
weed_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_2.jpg')
weed_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_3.jpg')

In [138]:
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_399.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_446.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_658.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_383.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_607.tif'
filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1150.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1151.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1141.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1136.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1137.tif'

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [weed_1, weed_2, weed_3], \
        [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (7,7), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 1, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.5, 2.6, 2.7, 2.8, 2.9, 3], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 175, morph_close = 10, \
        dilation_size = 301, \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)


# Export spray map to GeoTIFF.
output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*spray_region)

# Export weed map to GeoTIFF.
output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

# Export classification CI to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, classification_confidence)

In [82]:
# TODO: export json or yaml file of input parameterisations.

In [ ]:
file_pattern = '/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1, weed_2, weed_3], \
            [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (7,7), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 1, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.5, 2.75, 3], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 175, morph_close = 10, \
            dilation_size = 301, \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

In [20]:
def model_post_process(image_filename, \
    morph_open = 10, morph_close = 10, n_median_filter = 3, dilation_kernel_size = 5, dilate_iterations = 1):
    
    dataset = rasterio.open(image_filename)
    img_bw = dataset.read(1).copy()

    # Denoise. 
    # se1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_close, morph_close))
    # se2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_open, morph_open))

    # mask = cv2.morphologyEx(img_bw, cv2.MORPH_CLOSE, se1)
    # mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
    # mask = mask < 255    
    # img_bw *= mask
    
    if n_median_filter > 0:
        img_bw = cv2.medianBlur(img_bw, n_median_filter)
    
    # Dilate.
    kernel = np.ones((dilation_kernel_size, dilation_kernel_size), np.uint8)
    img_bw = cv2.dilate(img_bw, kernel, iterations=dilate_iterations)
    
    # Export classification map to GeoTIFF.
    output_filename = image_filename.replace('.tif',f'_SPRAY_REGION.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    _,img_binarized = cv2.threshold(img_bw, 254, 255, cv2.THRESH_BINARY)
    
    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, img_binarized)
    
    return

In [22]:
file_pattern = '/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/tiles/prev/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    model_post_process(filename, n_median_filter = 0, dilation_kernel_size = 500)

100_Orthomosaic_rgb_tile_616_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_765_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_179_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_583_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_878_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_85_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1080_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_431_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_542_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1041_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1132_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_44_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_258_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_444_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_537_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_31_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1034_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1147_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_485_CLASSIFICATION_MAP.tif
100_Orthom

100_Orthomosaic_rgb_tile_876_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_905_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_350_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_223_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_325_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_256_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_539_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1149_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_102_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_297_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1188_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_803_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_970_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_35_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1143_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1030_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_533_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_440_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1182_CLASSIFICATION_MAP.tif
100_Orth

100_Orthomosaic_rgb_tile_77_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1072_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1101_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_797_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_402_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_571_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_938_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_625_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_756_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1220_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_650_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_723_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1007_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1174_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_477_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_504_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_691_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_318_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1211_CLASSIFICATION_MAP.tif
100_Ort

100_Orthomosaic_rgb_tile_943_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_131_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1009_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_982_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_479_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_316_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_265_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_801_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_972_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_295_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_489_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_100_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1038_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_448_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_327_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_254_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_48_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_352_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_221_CLASSIFICATION_MAP.tif
100_Orthomo

100_Orthomosaic_rgb_tile_560_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_413_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1093_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_96_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_918_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_590_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_776_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_605_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_57_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1121_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1052_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_551_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_422_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1154_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_1027_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_22_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_524_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_457_CLASSIFICATION_MAP.tif
100_Orthomosaic_rgb_tile_338_CLASSIFICATION_MAP.tif
100_Orthom

RasterioIOError: /Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/tiles/prev/weed_map.tif: No such file or directory

In [8]:
def post_analyse(image_filename, weed_map_filename):
    
    image_dataset = rasterio.open(image_filename)
    red,green,blue = image_dataset.read(1), image_dataset.read(2), image_dataset.read(3)

    weed_map_dataset = rasterio.open(weed_map_filename)
    mask = weed_map_dataset.read(1)
    
    return np.extract(mask == 255, red), \
            np.extract(mask == 255, green), \
            np.extract(mask == 255, blue)

In [ ]:
band_red, band_green, band_blue = post_analyse(
'/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/100_Orthomosaic_rgb.tif',
'/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/model_output/weed_map.tif')

In [ ]:
fig, ax = plt.subplots()
plt.scatter(band_red, band_green)
plt.title(f'red, green')
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.scatter(band_blue, band_green)
plt.title(f'blue, green')
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.scatter(band_red, band_blue)
plt.title(f'red, blue')
plt.show()

In [67]:
model_post_process(
'/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/tiles/weed_map.tif',
morph_close = 15, morph_open = 60, \
n_median_filter = 11, \
dilation_kernel_size = 10, dilate_iterations = 4)

## nihill radish

Import test imagery:

In [22]:
# filename = 'DJI_20221105165501_0114_V'
# filename = 'DJI_20221105165324_0076_V'
# filename = 'DJI_20221105171447_0579_V'
# filename = 'DJI_20221105165900_0208_V'
# filename = 'DJI_20221105170156_0277_V'
# filename = 'DJI_20221105165643_0154_V'
# filename = 'DJI_20221105171530_0596_V'
# filename = 'DJI_20221105165306_0069_V'
# filename = 'DJI_20221105170235_0292_V'
# filename = 'DJI_20221105171310_0541_V'
# filename = 'DJI_20221105170743_0413_V'
# filename = 'DJI_20221105170442_0342_V'
# filename = 'DJI_20221105170447_0344_V'
# filename = 'DJI_20221105171333_0550_V'
# filename = 'DJI_20221105171507_0587_V'
# filename = 'DJI_20221105171510_0588_V'
# filename = 'DJI_20221105171512_0589_V'
# filename = 'DJI_20221105165033_0009_V'
# filename = 'DJI_20221105165843_0201_V'
# filename = 'DJI_20221105170513_0354_V'
# filename = 'DJI_20221105171436_0575_V'
filename = 'DJI_20221105171312_0542_V'
img2 = cv2.imread(f'test/{filename}.JPG')

In [26]:
image_norm = np.sum(img2, axis=2)

In [27]:
image_norm.shape

(3956, 5280)

Import features for detection: 

In [3]:
nihill_radish_sample_1 = cv2.imread('nihill_radish/nihill_radish_sample_1.jpg')
nihill_radish_sample_2 = cv2.imread('nihill_radish/nihill_radish_sample_2.jpg')
nihill_radish_sample_3 = cv2.imread('nihill_radish/nihill_radish_sample_3.jpg')
nihill_radish_sample_4 = cv2.imread('nihill_radish/nihill_radish_sample_4.jpg')
nihill_radish_sample_5 = cv2.imread('nihill_radish/nihill_radish_sample_5.jpg')
nihill_radish_sample_6 = cv2.imread('nihill_radish/nihill_radish_sample_6.jpg')
nihill_radish_sample_7 = cv2.imread('nihill_radish/nihill_radish_sample_7.jpg')
nihill_radish_sample_1.shape, nihill_radish_sample_2.shape, nihill_radish_sample_3.shape, \
nihill_radish_sample_4.shape, nihill_radish_sample_5.shape, nihill_radish_sample_6.shape, \
nihill_radish_sample_7.shape

((57, 102, 3),
 (69, 71, 3),
 (240, 74, 3),
 (97, 92, 3),
 (183, 106, 3),
 (47, 55, 3),
 (83, 89, 3))

In [4]:
wheat_nov_22_sample_1 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_sample_1.jpg')
wheat_nov_22_sample_2 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_sample_2.jpg')
wheat_nov_22_sample_3 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_sample_3.jpg')
wheat_nov_22_dead_sample_1 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_dead_sample_1.jpg')
wheat_nov_22_dead_sample_2 = cv2.imread('nihill_wheat_nov_22/nihill_wheat_nov_22_dead_sample_2.jpg')
wheat_nov_22_sample_1.shape, wheat_nov_22_sample_3.shape, \
wheat_nov_22_dead_sample_1.shape, wheat_nov_22_dead_sample_2.shape

((491, 182, 3), (213, 271, 3), (143, 170, 3), (620, 896, 3))

In [5]:
gravel_road_sample_1 = cv2.imread('gravel_road/gravel_road_sample_1.JPG')
gravel_road_sample_1.shape

(167, 561, 3)

In [ ]:
wheat_nov_22_dead_sample_2

In [ ]:
density, classification_image, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [nihill_radish_sample_1, nihill_radish_sample_2, nihill_radish_sample_3, \
        nihill_radish_sample_4, nihill_radish_sample_5, nihill_radish_sample_6, \
        nihill_radish_sample_7], \
        [wheat_nov_22_sample_1, wheat_nov_22_sample_2, wheat_nov_22_sample_3], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (17,17), \
        template_blur_kernel_size = (17,17), \
        template_resize_size = (32,32), \
        duplicate_tolerance = 5, \
        n_sigma_thresholds = [2.,2.25,2.5,2.75,3], \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)

cv2.imwrite(f'test/{filename}_CLASSIFICATION_MAP_VERSION_5.PNG', classification_image)

In [37]:
weed_map = cv2.imread('test/DJI_20221105170513_0354_V_CLASSIFICATION_MAP_VERSION_4.PNG', cv2.IMREAD_UNCHANGED)

In [ ]:
version_number = 5
plt.ioff()
# file_pattern = '/Users/user/Documents/Research/rotavision/model/test/*V.JPG'
file_pattern = '/Users/user/Documents/Research/rotavision/DJI_202211051646_002_DNihill-Raddish/*.JPG'
for filename in glob.glob(file_pattern):
    if os.path.exists(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG')):
        continue
    print(os.path.basename(filename))
    img = cv2.imread(filename)
    density, classification_image, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [nihill_radish_sample_1, nihill_radish_sample_2, nihill_radish_sample_3, \
            nihill_radish_sample_4, nihill_radish_sample_5, nihill_radish_sample_6, \
            nihill_radish_sample_7], \
            [wheat_nov_22_sample_1, wheat_nov_22_sample_2, wheat_nov_22_sample_3], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (17,17), \
            template_blur_kernel_size = (17,17), \
            template_resize_size = (32,32), \
            duplicate_tolerance = 5, \
            n_sigma_thresholds = [2.,2.25,2.5,2.75,3], \
            export_histogram = True, \
            export_heatmap = True, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)

    # Export classification map.
    cv2.imwrite(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG'), classification_image)
    
    # Copy EXIF data from original image to classification map.
    image_with_exif = Image.open(filename)
    exif = image_with_exif.info['exif']

    image_wo_exif = Image.open(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG'))
    image_wo_exif.save(filename.replace('.JPG',f'_CLASSIFICATION_MAP_VERSION_{version_number}.PNG'), 'PNG', exif=exif)

In [ ]:
show_bgra_image(classification_image, axis = 'on')

In [120]:
# Copy EXIF data from original image to classification map.
from PIL import Image
image_with_exif = Image.open('/Users/user/Documents/Research/rotavision/model/test/DJI_20221105165324_0076_V.JPG')
exif = image_with_exif.info['exif']

image_wo_exif = Image.open(\
'/Users/user/Documents/Research/rotavision/model/test/DJI_20221105165324_0076_V_CLASSIFICATION_MAP.PNG')

image_wo_exif.save(\
'/Users/user/Documents/Research/rotavision/model/test/DJI_20221105165324_0076_V_CLASSIFICATION_MAP_TEST.PNG', 
'PNG', exif=exif)